In [102]:
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry
import time

In [103]:
C = 0
V = 0

In [104]:
# Here we do not need a simulation world
# adding a configuration world
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")
V = ry.ConfigurationViewer()
V.setConfiguration(C)
X0 = C.getFrameState()

In [105]:
obj = C.addFrame("object")
obj.setPosition([-0.3,0.05,1.0])
obj.setQuaternion([1,0,0,0])
obj.setShape(ry.ST.ssBox, [0.1, 0.01, 0.2, .02])
obj.setColor([1,0,1])
V.setConfiguration(C)

In [106]:
R_gripper = C.frame("R_gripper")
L_gripper = C.frame("L_gripper")
R_gripper.setContact(1)
L_gripper.setContact(1)
obj.setContact(1)

In [107]:
C.getJointState()
q0_1,J = C.evalFeature(ry.FS.qItself, [])
print(q0_1)

[ 0. -1.  0. -2.  0.  2.  0.  0.  0. -1.  0. -2.  0.  2.  0.  0.]


In [108]:
#1.a

IK = C.komo_IK(False)

IK.addObjective([1.], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])
IK.addObjective(type=ry.OT.sos, feature=ry.FS.positionDiff, frames=['R_gripperCenter', 'L_gripperCenter'], scale=[1e3])
IK.addObjective(type=ry.OT.sos, feature=ry.FS.scalarProductZZ, frames=['R_gripperCenter', 'L_gripperCenter'], target=[-1])
IK.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductXX, frames=['R_gripperCenter', 'L_gripperCenter'])
IK.optimize(True)
C.setFrameState( IK.getConfiguration(0) )
V.setConfiguration(C) #to update your model display
#IK.getReport()

In [109]:
C.getJointState()
qT_1,J = C.evalFeature(ry.FS.qItself, [])
print(qT_1)

[-0.66607716 -0.8754499  -0.51910369 -1.70960973 -0.72183523  2.4028709
  0.03224742  0.00694541  0.67252373 -0.85044695  0.53468959 -1.70423721
  0.7202108   2.43329536 -0.03302998 -0.00603202]


In [110]:
#1.b

IK = C.komo_IK(False)


IK.addObjective([1.], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])

IK.addObjective(type=ry.OT.sos,   feature=ry.FS.positionDiff, frames=['R_gripperCenter', 'object'], scale=[1e3], target=[0.05,0,0])
IK.addObjective(type=ry.OT.sos,   feature=ry.FS.positionRel, frames=['L_gripperCenter', 'object'], scale=[1e3], target=[0,0,0.1])

IK.addObjective(type=ry.OT.eq,  feature=ry.FS.scalarProductXZ, frames=['object', 'R_gripperCenter'], target=[1])
IK.addObjective(type=ry.OT.eq,   feature=ry.FS.scalarProductXZ, frames=['R_gripperCenter', 'object'])


IK.addObjective(type=ry.OT.eq,   feature=ry.FS.scalarProductYY, frames=['L_gripperCenter', 'object'])
IK.addObjective(type=ry.OT.eq,   feature=ry.FS.scalarProductZZ, frames=['L_gripperCenter', 'object'], target=[1])

IK.optimize(True)
C.setFrameState( IK.getConfiguration(0) )
V.setConfiguration(C) #to update your model display
#IK.getReport()

In [111]:
C.getJointState()
qT_2,J = C.evalFeature(ry.FS.qItself, [])
print(qT_2)

[-4.36192048e-01 -3.11451870e-01  1.94972921e-01 -2.22555120e+00
  6.54857640e-02  1.92010499e+00  2.08413041e+00  1.77908011e-03
 -7.82368204e-02  1.52151077e+00  1.59098162e+00 -1.69271899e+00
  7.61884439e-01 -3.10174135e+00 -1.49625726e+00 -5.67590932e-04]


In [119]:
C.setFrameState(X0)
V.setConfiguration(C)

## Path Planning

In [113]:
#2.a,b

q0 = q0_1  
qT = qT_1 

#q0 = qT_1
#qT = qT_2

T = 5.0

for t in np.arange(0,T,0.1):
    qt = q0 + (0.5 *(1-np.cos(np.pi*t / T))*(qT - q0) )
    C.setJointState(qt)
    V.setConfiguration(C)
    time.sleep(0.1)

## Path Optimization

In [120]:
obj.setPosition([0.0,0,1.5])
V.setConfiguration(C)

In [121]:
# we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)

komo = C.komo_path(1.,20, 5., True) 

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])

komo.addObjective([1.], ry.FS.positionDiff, ["R_gripperCenter", "object"], ry.OT.sos, [1e2],target=[0.05,0,0])
komo.addObjective([1.], ry.FS.positionDiff, ["L_gripperCenter", "object"], ry.OT.sos, [1e2],target=[0,0,0.1])

komo.addObjective([1.], ry.FS.scalarProductXZ, ["object", "R_gripperCenter"], ry.OT.eq, target=[1])
komo.addObjective([1.], ry.FS.scalarProductXZ, ["R_gripperCenter", "object"], ry.OT.eq)

komo.addObjective([1.], ry.FS.scalarProductYY, ["L_gripperCenter", "object"], ry.OT.eq)
komo.addObjective([1.], ry.FS.scalarProductZZ, ["L_gripperCenter", "object"], ry.OT.eq, target=[1])

komo.addObjective(time=[1.], feature=ry.FS.qItself, type=ry.OT.eq, order=1);
komo.optimize(True)
komo.getReport()

[{'x_dim': 320, 'T': 20, 'k_order': 2, 'tau': 0.25, 'useSwift': True},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'QuaternionNorms',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'eq_sumOfAbs': 0.0},
 {'order': 2.0,
  'type': 'sos',
  'feature': 'qItself#32',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'sos_sumOfSqr': 1.1868847833371357},
 {'order': 0.0,
  'type': 'sos',
  'feature': 'qItself#32',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'sos_sumOfSqr': 0.0023761244243520924},
 {'order': 0.0,
  'type': 'ineq',
  'feature': 'ProxyCost',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'inEq_sumOfPos': 4.80041399929243e-06},
 {'order': 0.0,
  'type': 'sos',
  'feature': 'Default-0-posDiff-R_gripperCenter-object',
  'vars': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  'sos_sumOfSqr': 2.961593071235651e-05},
 {'order': 0.0,
  'type': 'sos',
  'feature': '

In [122]:
V = komo.view()

In [123]:
V.playVideo()

In [124]:
C.setFrameState(komo.getConfiguration(19))
C.getJointState()
V.setConfiguration(C)
coll = C.feature(ry.FS.accumulatedCollisions, [])
C.computeCollisions() 
coll.eval(C)

(array([4.800414e-08]),
 array([[ 0.09733172,  0.64493663,  0.06723702, -0.30989182,  0.03348602,
          0.02165985,  0.04504148,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]]))

In [125]:
C = 0
V = 0